In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import sys
import time
import threading
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser


from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-03-06 18:39:03,676 - Authenticating...
2023-03-06 18:39:04,237 - Successfully authenticated


In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [6]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

### Metrics

In [7]:
from api.watchdog.specs import get_trades, get_targets
from api.watchdog.pricer import Pricer

In [8]:
trades = get_trades(clarion)

In [9]:
targets = get_targets()

In [10]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

In [11]:
def highlight_cond(df, props, col):
    d = pd.DataFrame(None, df.index, df.columns)
    d.loc[df[col]] = props
    return d 

In [12]:
def highlight_target(df, props=None):
    d = pd.DataFrame(None, df.index, df.columns)
    for index, row in df.iterrows():
        props = 'background-color:red; color: Gold; font-weight: bold;' if row['breach'] else 'color: DarkOrange; font-weight: bold;'
        d.loc[index, ['target', row['var']]] = props
    return d

In [13]:
from importlib import reload
from api.watchdog import specs

In [14]:
def fn(pricer, out1, out2, send_email=True, email_freq=1, send_email_now=False, do_one=False):
    last_update = time.ctime()
    last_email_sent = time.time()
    p = pricer
    while True:
        if event.is_set():
            break
        try:
            reload(specs)
            from api.watchdog.specs import get_trades, get_targets
            trades = get_trades(clarion)
            targets = get_targets()
            p.get_data(runner, trades, targets, metrics=[*clarion_metrics.values()])
            clear_output()
            last_update = time.ctime()
            s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None) \
                .apply(highlight_target, axis=None) \
                .format(na_rep='') \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='-'), 
                    subset=p.watch.columns[p.watch.dtypes == 'float64'].drop('target')
                ) \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='0'), 
                    subset='target'
                ) \
                .set_caption('Trades and triggers')
            s1 = p.res.style.format(na_rep='').format(
                partial(millify, decimal=3, pct=None, zero_rep='-'), 
                subset=p.res.columns[p.res.dtypes == 'float64']
            )
            indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
            s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')
            s1 = s1.hide(subset=p.res.columns[p.res.columns.str.startswith('_')], axis=1).set_sticky(axis=1)
            # clear_output()
            # out.clear_output(wait=True)
            out1.outputs = ()
            out1.append_display_data(last_update)
            out1.append_display_data(s2)
            out2.outputs = ()
            out2.append_display_data(last_update)
            out2.append_display_data(s1)
            # display(out)
            if send_email:
                if (((time.time() - last_email_sent) / 3600) >= email_freq) or send_email_now:
                    if send_email_now:
                        send_email_now = False
                    print('sending')
                    outlook = win32.Dispatch('outlook.application')
                    mail = outlook.CreateItem(0)
                    mail.To = 'menashe.banit@brevanhoward.com'
                    # mail.To = 'giovambattista.perciaccante@brevanhoward.com'
                    mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
                    file_name = 'trades'
                    html_body = ''
                    for i, s in enumerate([s2, s1]):
                        file_name_ = f"{file_name}{i}.png"
                        dfi.export(s, file_name_, max_cols=-1)
                        image_file = open(file_name_, 'rb').read()
                        encoded_image = base64.b64encode(image_file).decode("utf-8")
                        html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
                        mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
                    mail.HTMLBody = html_body
                    # mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
                    # To attach a file to the email (optional)
                    # attachment  = "Path to the attachment"
                    mail.Send()
                    last_email_sent = time.time()
            # break
            if not do_one:
                time.sleep(30)
        except Exception as e:
            # return s1
            # raise e
            # clear_output()
            out.outputs = ()
            out.append_display_data(f"BROKEN: {last_update}")
            out.append_display_data(s2)
            out.append_display_data(s1)
            time.sleep(5)
        if do_one:
            break

In [15]:
p = Pricer()
out_summary = w.Output(layout={'overflow-x': 'auto'})
out_details = w.Output(layout={'overflow-x': 'auto', 'overflow-y': 'auto', 'height': '700px'})
event = threading.Event()

In [17]:
children = [out_summary, out_details]
tab = w.Tab(children=children, titles=['watchdog', 'details'])

In [18]:
thread = threading.Thread(target=fn, kwargs={
    'pricer': p, 
    'out1': out_summary, 
    'out2': out_details, 
    'send_email': True, 
    'email_freq': 1
})

In [19]:
s1 = fn(p, out_summary, out_details,  True, 1, True, do_one=True) 

sending


In [ ]:
thread.start()

### WATCH

In [ ]:
tab

In [ ]:
0

In [ ]:
event.set()

### Send EMAIL

In [ ]:
p.get_data(trades, targets)

In [20]:
s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None) \
    .apply(highlight_target, axis=None) \
    .format(na_rep='') \
    .format( 
        partial(millify, decimal=3, pct=None, zero_rep='-'), 
        subset=p.watch.columns[p.watch.dtypes == 'float64'].drop('target')
    ) \
    .format( 
        partial(millify, decimal=3, pct=None, zero_rep='0'), 
        subset='target'
    ) \
    .set_caption('Trades and triggers')
res_email = p.res.loc[:, ~p.res.columns.str.startswith('_')]
s1 = res_email.style.format(na_rep='').format(
    partial(millify, decimal=3, pct=None, zero_rep='-'), 
    subset=res_email.columns[res_email.dtypes == 'float64']
)
indices = p.res.loc[:, ~p.res.columns.str.startswith('_')].groupby('trade', sort=False).apply(lambda df: df.index[-1])
s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')

In [21]:
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

In [28]:
from_email = 'gioreports@brevanhoward.com'
smtp_server = "smtp.rivagecapital.com"
to_email = [
    'giovambattista.perciaccante@brevanhoward.com',
    # 'menashe.banit@brevanhoward.com'
]

message = MIMEMultipart("alternative")
message["Subject"] = f'Ciao'
message["From"] = from_email
message["To"] = ','.join(to_email)
html_ = MIMEText('Ciao', "html")
message.attach(html_)

smtp = smtplib.SMTP(smtp_server)
smtp.sendmail(from_email, to_email, message.as_string())

{}

In [61]:
from_email = 'gioreports@brevanhoward.com'
smtp_server = "smtp.rivagecapital.com"
to_email = [
    'giovambattista.perciaccante@brevanhoward.com',
    # 'menashe.banit@brevanhoward.com'
]

message = MIMEMultipart("alternative")
message["Subject"] = f'QUICK UPDATE - Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
message["From"] = from_email
message["To"] = ','.join(to_email)

In [62]:
html_body = ''
file_name = 'trades'
for i, s in enumerate([s2, s1]):
    file_name_ = f"{file_name}{i}.png"
    dfi.export(s, file_name_)
    image_data = open(file_name_, 'rb').read()
    encoded_image = base64.b64encode(image_data).decode("utf-8")
    html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
    image = MIMEImage(image_data, name=file_name_)
    # message.attach(image)
    
    # mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")

In [64]:
html_ = MIMEText(html_body, "html")

In [65]:
message.attach(html_)

In [66]:
smtp = smtplib.SMTP(smtp_server)
# smtp.set_debuglevel(1)
smtp.sendmail(from_email, to_email, message.as_string())

{}

In [ ]:
message.as_string()

In [ ]:
os.path.basename(str(Path().resolve() / file_name_))

In [ ]:
str(Path().resolve() / file_name_)

In [ ]:
file_name_

In [ ]:
html_ = MIMEText(html, "html")
message.attach(html_)

smtp = smtplib.SMTP(smtp_server)
# smtp.sendmail(from_email, to_email, message.as_string())

In [ ]:
s1

In [ ]:
s1.set_sticky(axis=1)

In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'menashe.banit@brevanhoward.com'
# mail.To = 'giovambattista.perciaccante@brevanhoward.com'
mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
file_name = 'trades'
html_body = ''
for i, s in enumerate([s2, s1]):
    file_name_ = f"{file_name}{i}.png"
    dfi.export(s, file_name_)
    image_file = open(file_name_, 'rb').read()
    encoded_image = base64.b64encode(image_file).decode("utf-8")
    html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
    mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
mail.HTMLBody = html_body
# mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
# To attach a file to the email (optional)
# attachment  = "Path to the attachment"
mail.Send()

In [ ]:
0